<a href="https://colab.research.google.com/github/SchmetterlingIII/D.T./blob/main/Bracing%20Concept/testing/curve_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intentions
**Prelude:**
I know how one can interpolate on splines normally but have no way of interpreting this information to give the feedback to a user that they are slouching.

From this initial intuition, I intend to develop my understanding of the spine, numerical methods in approaching that shape, and the demonstration of this happening in real time with the sensors.


## Notes
### Sensors
- IMU noise clean up (Kalman Filtering*)
- IMU placement
- Spine model verification
- Planning basic layout of haptic motor set up

The IMU placement needs to be very specific to certain vertebrae (or at least that region) of the vertebral column. The anchoring points of the vertebrae (e.g. the inflection points of curvature on the spine or areas with a relatively consistent/definable curvature) would be useful for the application of the sensor as, at those points, conditions for the spline equations can be made.

The verification of displayed results (and some methodology for this) is quite a difficult result to achieve and **need to work on finding some methodology to test the error of these measurements (and then I could design some way of minimising this cost function)**.

**Initially just visualise the noise and experiment personally on methods of denoising (and get an intuition for the Kalman Filter when it is actually applied).*

### Local Interpretation
*Can there be a global function? I feel like I would just have to use the array of changing local 2nd derivatives and say that this is the "global" function since it is much more efficient than trying to learn some way of applying this when more time can be spent on the human-centered side of it and I want to spend less time on the technical stuff.*

- Tutorials for collections.deque:
  https://youtu.be/cVqRJIAr7Ls?si=Ridx5prT2bkguSvG
  https://youtu.be/m3JgSV1Obn8?si=9hMpTtcbi3aWGLLy
  *Intermediate Python Tutorials* by *Tech with Tim*

- *How to properly define the parameters for each section of the spine to curve in a manner akin to a human being. Will torsion (mathematically) play a role for the simple version of this to be completed?*

- *How the checking of the collections.deque databases will work (stored only across a certain timespan for the D.T. project and can be expanded later) and the checking of the second derivative from that.*

For the global measurement of the spine (as a way of optimising the flow of code), measuring how much second derivative changes wrt time will indicate whether there is tilting of any degree across the function.

The iterative nature of reading the second derivative (wrt time) informs whether I should iterate across each of the spline segments to check for where the deviation (specifically) has occurred. This will be done by checking each of the databases (**stored under collections.deque**) underneath each of these functions.

The deviated function would be recognised, and a signal will be sent to the serial (back to the microcontroller) for the haptic motor to be activated.

**Need to understand how this haptic motor will be activated next + the more practical sides of this design. And quickly.**
  

In [ ]:
"""
PSEUDOCODE:
**Send the emails sooner rather than later and get this breezed through now since the designing aspect is the more difficult part.**
1-4 MUST BE DONE BEFORE SCHOOL. NEED TO WORK FASTER!!
**start with 2/3 IMUs and then progress from there**

1. Extracting vectors from serial data and plotting them
2. Seeing the produced splines and (only by hand), understanding the shape they form
    a. if switching to b-splines at this point, having a good intuition for how parameters can change the shape of the curve
    b. potentially de-noising and spending time to understand that (but minor since big picture is the design of this)
3. Using the functions of each of the sections, use sympy to get the second derivative and place that into a dynamic collections.deque (having learnt how to use it)
    a. then, I can set timers that e.g. if f''(x) > 0 for 10 seconds, ax.plot(..., color='r', ...) as some input output
    b. progress to changing the LEDs on the Arduino to blink at a distinct frequency (to reinforce this communication)
4. At this point, I will spend much more time on the aesthetics and making sure that this can be a sold device
    a. Design is crucial (although quite simple, getting a working prototype is harder so get the theory done well)
    b. research into the more complex stuff (and write-ups and methodologies on how to verify this concept in the field)
    c. Emailling orthopaedic surgeons here (since now I have proper data and github repositories, do this through LinkedIn if need be) for further help on this
"""

In [5]:
import sympy as sym
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

In [6]:
# create random functions that iterate between two points
x = np.random.uniform(-5, 5, 10)
y = np.random.uniform(-5, 5, 10)

func = interp1d(x,y,kind='cubic')
print(func)

In [10]:
import sympy as sym
import numpy as np
from scipy.interpolate import CubicSpline
np.set_printoptions(precision=10)

x = np.random.uniform(-5, 5, 10)
y = np.random.uniform(-5, 5, 10)

# Create the cubic spline object
cs = CubicSpline(x, y)

# Get the coefficients for each polynomial segment
# cs.c is a 4x(n-1) array where n is the number of points.
# The rows correspond to coefficients of x^3, x^2, x^1, and x^0
coefficients = cs.c

# Now, use sympy to create the symbolic equation for each segment
x_sym = sym.Symbol('x')

# Let's get the equation for the first segment (between x[0] and x[1])
# The coefficients are ordered as [c3, c2, c1, c0]
c3, c2, c1, c0 = coefficients[:, 0]

# Create the symbolic polynomial
poly_segment_0 = c3 * (x_sym - x[0])**3 + c2 * (x_sym - x[0])**2 + c1 * (x_sym - x[0]) + c0

# Display the equation
print(f"Equation for the first segment (between x={x[0]} and x={x[1]}):")
sym.pprint(poly_segment_0.expand())

ValueError: `x` must be strictly increasing sequence.

## Research
*This is very secondary stuff to the work that I intend to do. My focus needs to be defined but it is not on spinal deformities yet*.

- Cobb Angle
  - what is it
  - how can its mathematical methods be used to help with the modelling of people with spinal curvatures

  https://pmc.ncbi.nlm.nih.gov/articles/PMC6399566/#abstract1: This paper explores the use of an automated determination of Cobb angles (w/ X-rays however) using a convolutional neural network (CNN). I would like to do something similar where a matrix map of the users back is determined through the sensors (later on) and can help to be an indication for spinal deformities (scoliosis is quite evident, moreso the development of turtleneck syndrome of end-of-life conditions). Neural nets will help with this aim.

- Kalman Filter

  https://cdn.ima.org.uk/wp/wp-content/uploads/2024/09/Kalman-Filter-Tutorial-Part-1_Mathematics-Today_2016.pdf


# Curve Fitting
Specifically will interpret all data points to fit the data onto what a spine actually looks like, so that the results are more useful.

# Direct Next Steps:
*I am going to need to test this with real sensors now and try to validate my findings. As tests, I can set relatively abitary curves on the points to see how they react in real time as well as:
  - Kalman filtering (and finding some ways of improving approximations)
  - Tests on the visualisation of this live (and whether to use non-matplotlib files to continue)

**What about maths?**

*This can be tested with pseudocode to test my inherent understanding of functions and learning function approximations*
- Segment the dynamic code and store the changes in the second derivative of the local cubic splines
- Produce a dynamic approximation for the global cubic spline


- reading in the IMU data (and getting a coherent mix between the data points (of gyroscope data)).
- specific pseudocode of the flow of the code (which can be written now)